In [8]:
#pip install schedule

In [9]:
import pandas as pd
import shutil, os, glob
import pymongo
from pymongo import MongoClient
import json
import schedule
import time
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


## Extract

In [10]:
df = pd.concat(map(pd.read_csv, glob.glob('input' + '\*.csv')))
#Extract All source code
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [11]:
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Magnitude,ID
0,1965-01-02 00:00:00+00:00,13:44:18,19.246,145.616,Earthquake,131.6,6.0,ISCGEM860706
1,1965-01-04 00:00:00+00:00,11:29:49,1.863,127.352,Earthquake,80.0,5.8,ISCGEM860737
2,1965-01-05 00:00:00+00:00,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,ISCGEM860762
3,1965-01-08 00:00:00+00:00,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,ISCGEM860856
4,1965-01-09 00:00:00+00:00,13:32:50,11.938,126.427,Earthquake,15.0,5.8,ISCGEM860890


In [ ]:
## Generate random latitude and longitudes
# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

# List for holding lat_lngs tuples
#lat_lng_rand = []

# Create a set of random lat and lng combinations
#lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
#lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)



lat_lng_rand = zip(lats, lngs)

## Generate cities list from random latitudes and longitudes using citipy.nearest_city
# List for holding cities
cities = []

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lng_rand:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [7]:
len(df)

23412

## Transform Hash

In [18]:
df['hash'] = pd.Series((hash(tuple(row)) for _, row in df.iterrows()))
#Hash Code
#https://stackoverflow.com/questions/25757042/create-hash-value-for-each-row-of-data-with-selected-columns-in-dataframe-in-pyt
df.tail()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,hash
542,542,scalea,36,IT,1528905631,100,39.82,15.79,70.06,5.30,-7776889599612997735
543,543,san andres,68,PH,1528905632,100,13.32,122.68,83.47,22.64,6153222264861819892
544,544,muisne,92,EC,1528905633,84,0.61,-80.02,76.90,4.63,-4358575719749853676
545,545,ati,20,TD,1528905633,25,13.21,18.34,104.53,4.97,668237472065734703
546,546,san luis,0,AR,1528905472,53,-33.30,-66.34,42.43,2.62,-3731135503255177272


In [19]:
len(df)

1095

## Transform Dedupe

In [20]:
df_nodupe = df[~df.duplicated()]
#Dedupe code source
#https://stackoverflow.com/questions/40438237/assign-hash-to-row-of-categorical-data-in-pandas

In [21]:
df_nodupe.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,hash
0,0,jacareacanga,0,BR,1528902000,62,-6.22,-57.76,89.60,6.93,8089838516891515384
1,1,kaitangata,100,NZ,1528905304,94,-46.28,169.85,42.61,5.64,2367684193594300934
2,2,goulburn,20,AU,1528905078,91,-34.75,149.72,44.32,10.11,-5985903078597179477
3,3,lata,76,IN,1528905305,89,30.78,78.62,59.89,0.94,-1873370033211715858
4,4,chokurdakh,0,RU,1528905306,88,70.62,147.90,32.17,2.95,8460587636036003065


In [22]:
len(df_nodupe)

547

## Load

#### > Convert DataFrame to json formatted file

In [23]:
import json

In [24]:
df_nodupe.to_json('ETL_json_file', orient='index')
# Converting dataframe data to json format
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html

In [25]:
json_data = df_nodupe.to_json(orient='index')
parsed = json.loads(json_data)
# print(json.dumps(parsed, indent=4, sort_keys=True))
# Un-comment print to pretty print json formated data

#### > Create MongoDB named 'etl_db' with a collection 'etl_data'

In [26]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
print('Mongo version', pymongo.__version__)

Mongo version 3.9.0


In [27]:
# Uncomment code to drop existing database before creating new one
# client.drop_database('etl_db')

In [28]:
db = client['etl_db']
# Create database named 'etl_db'

db.etl_data.drop() # Drop collection if it already exist

collection = db['etl_data']
# Create collection called 'etl_data'

#### > Load ETL_json_file data

In [29]:
with open('ETL_json_file') as f:
    file_data = json.load(f)
    collection.insert_one(file_data) 
client.close()
# Read json file into mongoDB
# https://stackoverflow.com/questions/49510049/how-to-import-json-file-to-mongodb-using-python

In [30]:
db.list_collection_names()
# List collections in database

['etl_data']

In [31]:
db.etl_data.find( { 0: {} } )


In [32]:
# cursor = db.etl_data # selecting the etl_data collection
# for document in cursor.find():
#    print (document)
# Printing the content of the collection LARGE PRINTOUT

#### > Reading content of database and loading it into a dataframe

In [33]:
extracted_etl_data = db.etl_data
df = pd.DataFrame(list(extracted_etl_data.find()))
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas

In [34]:
df

,0,1,10,100,101,102,103,104,105,106,...,91,92,93,94,95,96,97,98,99,_id
0,"{'City_ID': 0, 'City': 'jacareacanga', 'Cloudi...","{'City_ID': 1, 'City': 'kaitangata', 'Cloudine...","{'City_ID': 10, 'City': 'puerto ayora', 'Cloud...","{'City_ID': 100, 'City': 'auchel', 'Cloudiness...","{'City_ID': 101, 'City': 'bubaque', 'Cloudines...","{'City_ID': 102, 'City': 'khatanga', 'Cloudine...","{'City_ID': 103, 'City': 'los banos', 'Cloudin...","{'City_ID': 104, 'City': 'labuhan', 'Cloudines...","{'City_ID': 105, 'City': 'saint-paul', 'Cloudi...","{'City_ID': 106, 'City': 'petatlan', 'Cloudine...",...,"{'City_ID': 91, 'City': 'mataura', 'Cloudiness...","{'City_ID': 92, 'City': 'buala', 'Cloudiness':...","{'City_ID': 93, 'City': 'eyl', 'Cloudiness': 2...","{'City_ID': 94, 'City': 'abu dhabi', 'Cloudine...","{'City_ID': 95, 'City': 'tahe', 'Cloudiness': ...","{'City_ID': 96, 'City': 'chuy', 'Cloudiness': ...","{'City_ID': 97, 'City': 'dzerzhinskoye', 'Clou...","{'City_ID': 98, 'City': 'vanavara', 'Cloudines...","{'City_ID': 99, 'City': 'muros', 'Cloudiness':...",5d7039a9ae8f05cc6640f391


In [35]:
df = df.melt()

In [36]:
df.head()

,variable,value
0,0,"{'City_ID': 0, 'City': 'jacareacanga', 'Cloudi..."
1,1,"{'City_ID': 1, 'City': 'kaitangata', 'Cloudine..."
2,10,"{'City_ID': 10, 'City': 'puerto ayora', 'Cloud..."
3,100,"{'City_ID': 100, 'City': 'auchel', 'Cloudiness..."
4,101,"{'City_ID': 101, 'City': 'bubaque', 'Cloudines..."


In [37]:
len(df)

548

In [38]:
df.iloc[0][1]
# Showing content of first row 'value'

{'City_ID': 0,
 'City': 'jacareacanga',
 'Cloudiness': 0,
 'Country': 'BR',
 'Date': 1528902000,
 'Humidity': 62,
 'Lat': -6.22,
 'Lng': -57.76,
 'Max Temp': 89.6,
 'Wind Speed': 6.93,
 'hash': 8089838516891515384}

#### > CSV Move script - moves .csv files after being ETL processed

In [39]:
import shutil, os, glob

In [40]:
#For Loop moving all .csv files from the test_input folder 
for filePath in glob.glob('input' + '\*.csv'):
            #Move each file to the destination directory
        shutil.move(filePath, 'processed');
# Move all files that have been ETL'ed into a processed folder
#Source Code           
#https://thispointer.com/python-how-to-move-files-and-directories/

##Schedule ETL Jobs

In [29]:
#pip install schedule

#Source Code
#https://github.com/dbader/schedule
import schedule
import time

def job():
    print("Performing an ETL process...")

schedule.every(1).minutes.do(job)

while 1:
    schedule.run_pending()
    time.sleep(1)